In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import tensorflow_text as text
import tensorflow_hub as hub
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from sklearn.model_selection import train_test_split
import re


In [2]:
df=pd.read_csv('train.csv')


In [3]:
df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [4]:
columns_to_drop = ['id', 'keyword', 'location']
df = df.drop(columns=columns_to_drop)

In [5]:
df

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chinmay\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chinmay\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chinmay\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
import zipfile
import os

# Define the path to the zip file and the output directory
zip_file_path = 'C:\\Users\\Chinmay\\AppData\\Roaming\\nltk_data\\corpora\\wordnet.zip'
output_dir = 'C:\\Users\\Chinmay\\AppData\\Roaming\\nltk_data\\corpora'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)


In [8]:
def preprocess_text(text):
    text = text.lower()
    tokens = word_tokenize(text)
    alphanumeric_tokens = [token for token in tokens if token.isalnum()]
    filtered_tokens = [token for token in alphanumeric_tokens if token not in stopwords.words('english') and token not in string.punctuation]
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return ' '.join(lemmatized_tokens)

In [9]:
df['processed_text'] = df['text'].apply(preprocess_text)


In [10]:
df

,text,target,processed_text
0,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake may allah forgive u
1,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,All residents asked to 'shelter in place' are ...,1,resident asked place notified officer evacuati...
3,"13,000 people receive #wildfires evacuation or...",1,people receive wildfire evacuation order calif...
4,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby alaska smoke wildfire pour...
...,...,...,...
7608,Two giant cranes holding a bridge collapse int...,1,two giant crane holding bridge collapse nearby...
7609,@aria_ahrary @TheTawniest The out of control w...,1,thetawniest control wild fire california even ...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utc 5km volcano hawaii http
7611,Police investigating after an e-bike collided ...,1,police investigating collided car little portu...


In [12]:
drp = ['text', 'target']
Graph_df = df.drop(columns = drp)

In [14]:
Graph_df

,processed_text
0,deed reason earthquake may allah forgive u
1,forest fire near la ronge sask canada
2,resident asked place notified officer evacuati...
3,people receive wildfire evacuation order calif...
4,got sent photo ruby alaska smoke wildfire pour...
...,...
7608,two giant crane holding bridge collapse nearby...
7609,thetawniest control wild fire california even ...
7610,utc 5km volcano hawaii http
7611,police investigating collided car little portu...


In [29]:
X = df['processed_text']
y = df['target']

# Tokenize the text 
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)

# Convert text to sequences of integers
X_sequences = tokenizer.texts_to_sequences(X)


max_length = max(map(len, X_sequences))
X_padded = tf.keras.preprocessing.sequence.pad_sequences(X_sequences, 
                                                         maxlen=max_length, padding='post')

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim 
                              =len(tokenizer.word_index) + 1, output_dim=128, 
                                                                  input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model with a specified learning rate
learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_padded, y, epochs=5, batch_size=500, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_padded, y)
print(f'Test accuracy: {accuracy:.4f}')

Epoch 1/5
13/13 [==============================] - 10s 309ms/step - loss: 0.6540 - accuracy: 0.6440 - val_loss: 0.6527 - val_accuracy: 0.6907
Epoch 2/5
13/13 [==============================] - 1s 90ms/step - loss: 0.3297 - accuracy: 0.8801 - val_loss: 0.5343 - val_accuracy: 0.7564
Epoch 3/5
13/13 [==============================] - 2s 140ms/step - loss: 0.1196 - accuracy: 0.9606 - val_loss: 0.5607 - val_accuracy: 0.7544
Epoch 4/5
13/13 [==============================] - 1s 94ms/step - loss: 0.0696 - accuracy: 0.9773 - val_loss: 0.6828 - val_accuracy: 0.7485
Epoch 5/5
238/238 [==============================] - 1s 3ms/step - loss: 0.1695 - accuracy: 0.9379
Test accuracy: 0.9379


In [12]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 23, 128)           1886720   
                                                                 
 flatten (Flatten)           (None, 2944)              0         
                                                                 
 dense (Dense)               (None, 128)               376960    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2

In [18]:
df1 = pd.read_csv('BIAtest.csv')


In [19]:
df1

,Id,keyword,Descreption
0,1.0,accident,Accident cleared in #PaTurnpike on PATP EB bet...
1,2.0,accident,Just got to love burning your self on a damn c...
2,3.0,accident,I hate badging shit in accident
3,4.0,accident,#3: Car Recorder ZeroEdgeå¨ Dual-lens Car Came...
4,5.0,accident,Coincidence Or #Curse? Still #Unresolved Secre...
...,...,...,...
675,NaN,NaN,NaN
676,NaN,NaN,NaN
677,NaN,NaN,NaN
678,NaN,NaN,NaN


In [20]:
columns_to_drop = [ 'keyword']
df1 = df1.drop(columns=columns_to_drop)
df1 = df1.dropna(subset=['Id'])

In [21]:
df1

,Id,Descreption
0,1.0,Accident cleared in #PaTurnpike on PATP EB bet...
1,2.0,Just got to love burning your self on a damn c...
2,3.0,I hate badging shit in accident
3,4.0,#3: Car Recorder ZeroEdgeå¨ Dual-lens Car Came...
4,5.0,Coincidence Or #Curse? Still #Unresolved Secre...
...,...,...
594,595.0,Rly tragedy in MP: Some live to recount horror...
595,596.0,Rly tragedy in MP: Some live to recount horror...
596,597.0,I've spent the day traumatised about the fact...
597,598.0,Can you imagine how traumatised Makoto would b...


In [23]:
df1['processed_text'] = df1['Descreption'].apply(preprocess_text)


In [25]:
df1

,Id,Descreption,processed_text
0,1.0,Accident cleared in #PaTurnpike on PATP EB bet...,accident cleared paturnpike patp eb cranberry ...
1,2.0,Just got to love burning your self on a damn c...,got love burning self damn curling wand swear ...
2,3.0,I hate badging shit in accident,hate badging shit accident
3,4.0,#3: Car Recorder ZeroEdgeå¨ Dual-lens Car Came...,3 car recorder car camera vehicle camcorder la...
4,5.0,Coincidence Or #Curse? Still #Unresolved Secre...,coincidence curse still unresolved secret past...
...,...,...,...
594,595.0,Rly tragedy in MP: Some live to recount horror...,rly tragedy mp live recount horror saw coach t...
595,596.0,Rly tragedy in MP: Some live to recount horror...,rly tragedy mp live recount horror saw coach t...
596,597.0,I've spent the day traumatised about the fact...,spent day traumatised fact load good music pro...
597,598.0,Can you imagine how traumatised Makoto would b...,imagine traumatised makoto would could see dub...


In [24]:
X1= df1['processed_text']


In [26]:
X1_sequences = tokenizer.texts_to_sequences(X1)


In [27]:
X_padded1 = tf.keras.preprocessing.sequence.pad_sequences(X1_sequences, maxlen=max_length, padding='post')


In [28]:
predictions = model.predict(X_padded1)

# Convert predictions to binary values (0 or 1)
binary_predictions = [1 if pred > 0.5 else 0 for pred in predictions]

# Print the binary predictions
print(binary_predictions)

19/19 [==============================] - 2s 11ms/step
[1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 